# Finetuning을 통해 분류 성능 올리기 - 데이터 준비하기


## 데이터 생성 Chain 만들기

In [1]:
from typing import List
import random
import json

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from tqdm.auto import tqdm

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
model = ChatOpenAI(model='gpt-4-turbo-preview', temperature=1.)

In [3]:
class Turn(BaseModel):
    name: str = Field(description="이름")
    content: str = Field(description="발화")

In [4]:
class Conversation(BaseModel):
    subject: str = Field(description="대화 주제")
    turn_list: List[Turn] = Field(description="subject와 sentiment를 기반으로한 소개팅 상황에서의 대화. tunr_list의 최대 길이는 4.")

In [5]:
parser = JsonOutputParser(pydantic_object=Conversation)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"subject": {"title": "Subject", "description": "대화 주제", "type": "string"}, "turn_list": {"title": "Turn List", "description": "subject와 sentiment를 기반으로한 소개팅 상황에서의 대화. tunr_list의 최대 길이는 4.", "type": "array", "items": {"$ref": "#/definitions/Turn"}}}, "required": ["subject", "turn_list"], "definitions": {"Turn": {"title": "Turn", "type": "object", "properties": {"name": {"title": "Name", "description": "이름", "type": "string"}, "content": {"title": "Content", "description": "발화", "type": "string"}}, "required": ["name", "c

In [6]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "두명의 한국인의 소개팅 상황에서의 대화를 만들어줘\n대화의 감정: {sentiment}\n{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template
    ])
prompt = prompt.partial(format_instructions=format_instructions)

In [7]:
conv_gen_chain = prompt | model | parser

In [8]:
conv_gen_chain.invoke({"sentiment": "긍정"})

{'subject': '취미와 관심사 공유하기',
 'turn_list': [{'name': '지훈',
   'content': '안녕하세요, 지훈입니다. 취미로 사진 찍는 걸 정말 좋아해요. 혹시 취미가 있으신가요?'},
  {'name': '소영',
   'content': '안녕하세요, 소영이에요. 저는 여행 다니면서 새로운 문화를 체험하는 걸 좋아해요. 사진 찍는 것도 여행 중 꼭 하는 건데, 우리 취미가 잘 맞는 것 같아요!'},
  {'name': '지훈', 'content': '정말요? 그럼 다음에 같이 사진 찍으러 다녀보는 건 어떨까요?'},
  {'name': '소영', 'content': '좋아요! 기대됩니다.'}]}

In [9]:
n_sample = 20

In [10]:
sample_list = []

for _ in tqdm(range(n_sample), total=n_sample):
    try:
        sentiment = "긍정" if random.random() < 0.5 else "부정"
        
        out = conv_gen_chain.invoke({"sentiment": sentiment})
        out['sentiment'] = sentiment
        sample_list.append(out)
    except Exception as e:
        print(e)

  0%|          | 0/20 [00:00<?, ?it/s]

'NoneType' object does not support item assignment


In [11]:
len(sample_list)

19

In [12]:
sample_list[3]

{'subject': '취미 공유하기',
 'turn_list': [{'name': '지훈',
   'content': '안녕하세요, 지훈이에요. 요즘에는 어떤 취미로 시간을 보내세요?'},
  {'name': '하은',
   'content': '안녕하세요, 하은입니다. 저는 요즘에 사진 찍는 걸로 취미 생활을 하고 있어요. 지훈 씨는요?'},
  {'name': '지훈',
   'content': '오, 사진 찍기 정말 멋진 취미네요. 저는 등산을 좋아해요. 자연을 느끼면서 운동도 되니까요.'},
  {'name': '하은',
   'content': '등산이요? 저도 가끔 산책 겸 가벼운 등산을 즐겨요. 같이 등산 가기도 좋을 것 같네요.'}],
 'sentiment': '긍정'}

## OpenAI finetuning 스타일 포맷 변경

- jsonline 포맷
- 예시
```json
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
```

In [13]:
def cvt_turn_list_to_conv_text(turn_list):
    conv_text = ""
    
    for turn in turn_list:
        conv_text += f"{turn['name']}: {turn['content']}\n"
    return conv_text.rstrip()

In [14]:
def sample_to_messages(sample):
    conv_text = cvt_turn_list_to_conv_text(sample['turn_list'])
    sentiment = sample['sentiment']
    
    msgs = {"messages": [{"role": "system", "content": "다음 대화의 내용에 대해 '긍정' 또는 '부정'으로 감성 분석해줘."},
                          {"role": "user", "content": conv_text},
                          {"role": "assistant", "content": sentiment}
                         ]}
    return msgs

In [15]:
# JSONL 파일 생성 함수
def create_jsonl(msgs_list, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for msgs in msgs_list:
            json_line = json.dumps(msgs, ensure_ascii=False)
            f.write(json_line + '\n')

In [16]:
sample_list[0]

{'subject': '취미 공유',
 'turn_list': [{'name': '지훈', 'content': '안녕하세요, 만나서 반가워요. 취미가 뭐에요?'},
  {'name': '은지', 'content': '안녕하세요! 저는 등산을 좋아해요. 지훈 씨는요?'},
  {'name': '지훈', 'content': '저도 자연을 좋아해서 종종 등산하곤 해요. 같은 취미를 가지고 있어서 반가워요.'},
  {'name': '은지', 'content': '정말요? 같이 등산하기 좋은 곳으로 가보고 싶네요.'}],
 'sentiment': '긍정'}

In [17]:
msgs_list = [sample_to_messages(sample) for sample in sample_list]

In [18]:
msgs_list[0]

{'messages': [{'role': 'system',
   'content': "다음 대화의 내용에 대해 '긍정' 또는 '부정'으로 감성 분석해줘."},
  {'role': 'user',
   'content': '지훈: 안녕하세요, 만나서 반가워요. 취미가 뭐에요?\n은지: 안녕하세요! 저는 등산을 좋아해요. 지훈 씨는요?\n지훈: 저도 자연을 좋아해서 종종 등산하곤 해요. 같은 취미를 가지고 있어서 반가워요.\n은지: 정말요? 같이 등산하기 좋은 곳으로 가보고 싶네요.'},
  {'role': 'assistant', 'content': '긍정'}]}

In [19]:
n_total = len(msgs_list)
n_train = int(n_total * 0.6)
n_valid = int(n_total*0.2)

In [20]:
train_msgs_list = msgs_list[:n_train]
valid_msgs_list = msgs_list[n_train:n_train + n_valid]
test_msgs_list = msgs_list[n_train + n_valid:]


In [21]:
len(train_msgs_list), len(valid_msgs_list), len(test_msgs_list)

(11, 3, 5)

In [22]:
# 훈련 및 검증 데이터셋을 JSONL 파일로 변환
create_jsonl(train_msgs_list, 'conv_sent_train.jsonl')
create_jsonl(valid_msgs_list, 'conv_sent_valid.jsonl')
create_jsonl(test_msgs_list, 'conv_sent_test.jsonl')